### Assigning trains to routes

The example below uses Google OR-tools to assign trains to routes. This is similar to the employee schedule problem found here: https://developers.google.com/optimization/scheduling/employee_scheduling

The example shows how to assign 6 trains to 8 routes subject to the following constraints:

- Each train must be used during the day and each route must be covered

- A train can be assigned to a maximum of two routes per day

- Where a train is assigned to two routes, the train's end of day cumulative kilometreage must not exceed 24,800 km

- Where a train is assigned to two routes the timings of the individual routes must not overlap
     
- Trains with high cumulative kilometreage should be assigned to short routes and trains with low cumulative kilometreage should be assigned to long routes



In [3]:
# Copyright 2010-2018 Google LLC 
 # Licensed under the Apache License, Version 2.0 (the "License"); 
 # you may not use this file except in compliance with the License. 
 # You may obtain a copy of the License at 
 # 
 #     http://www.apache.org/licenses/LICENSE-2.0 
 # 
 # Unless required by applicable law or agreed to in writing, software 
 # distributed under the License is distributed on an "AS IS" BASIS, 
 # WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. 
 # See the License for the specific language governing permissions and 
 # limitations under the License. 


from itertools import combinations
from ortools.sat.python import cp_model
 

    
# define a function which tests whether a pair of times overlaps - 
# returns true if they do, else false
def test_overlap(t1_st, t1_end, t2_st, t2_end):
    
    def convert_to_minutes(t_str):
        hours, minutes = t_str.split(':')
        return 60*int(hours)+int(minutes)

    t1_st = convert_to_minutes(t1_st)
    t1_end = convert_to_minutes(t1_end)
    t2_st = convert_to_minutes(t2_st)
    t2_end = convert_to_minutes(t2_end)

# Check for wrapping time differences
    if t1_end < t1_st:
        if t2_end < t2_st:
        # Both wrap, therefore they overlap at midnight
            return True
        # t2 doesn't wrap. Therefore t1 has to start after t2 and end before
        return t1_st < t2_end or t2_st < t1_end

    if t2_end < t2_st:
        # only t2 wraps. Same as before, just reversed
        return t2_st < t1_end or t1_st < t2_end

    # They don't wrap and the start of one comes after the end of the other,
    # therefore they don't overlap
    if t1_st >= t2_end or t2_st >= t1_end:
        return False
    # In all other cases, they have to overlap
    return True



def main():
    model = cp_model.CpModel()
    solver = cp_model.CpSolver()
 
    # length of the routes
    route_km = {
        'R11': 700,
        'R32': 600,
        'R16': 600,
        'R41': 10,
        'R42': 10,
        'R43': 10,
        'R44': 10,
        'R45': 10}
    
    # previous day cumulative km for each train
    train_cum_km = {
        'T32': 24_300,
        'T11': 24_300,
        'T38': 24_300,
        'T28': 600,
        'T15': 200,
        'T24': 100}
    
    # start and end times for each route
    route_times = {
        'R11': ('05:00', '00:00'),
        'R32': ('06:00', '00:50'),
        'R16': ('05:20', '23:40'),
        'R41': ('11:15', '12:30'),
        'R42': ('11:45', '13:00'),
        'R43': ('12:15', '13:30'),
        'R44': ('12:45', '14:00'),
        'R45': ('13:20', '14:35')}
    
    
    
    trains = list(train_cum_km.keys())
    routes = list(route_km.keys())
    num_trains = len(trains)
    num_routes = len(routes)
    
    # this array defines assignments for routes to trains where assignments[(t,r)] equal 1 
    # if route r is assigned to train t , else it's 0
    assignments = {(t, r): model.NewBoolVar('assignment_%s%s' % (t, r))
                   for t in trains for r in routes}
    
    
    # constraint 1: each train must be used
    for r in routes:
        model.Add(sum(assignments[(t, r)] for t in trains) == 1)
 
    # constraint 2: each train must do at least one (max two) routes
    for t in trains:
        model.Add(sum(assignments[(t, r)] for r in routes) >= 1)
        model.Add(sum(assignments[(t, r)] for r in routes) <= 2)
 
    
    # constraint 3: ensure the end of day cum km is less than 24_800
    # create a new variable which must be in the range (0,24_800)
    day_end_km = {
        t: model.NewIntVar(0, 24_800, 'train_%s_day_end_km' % t)
        for t in trains
    }
    
    for t in trains:
        # this will be constrained because day_end_km[t] is in domain [0, 24_800]
        tmp = sum(assignments[t, r]*route_km[r] for r in routes) + train_cum_km[t]   
        model.Add(day_end_km[t] == tmp)
 
    
    # constraint 4: where 2 routes are assigned to a train, these must not overlap
    for (r1, r2) in combinations(routes, 2):
            if test_overlap(route_times[r1][0], route_times[r1][1], route_times[r2][0], route_times[r2][1]):
                 for train in trains:
                    model.AddBoolOr([assignments[train, r1].Not(), assignments[train, r2].Not()])
    
    # constraint 5: trains with high cum kilometreage must do short routes, trains with low cum kilometreage
    #               must do long routes
    
    score = {
             (t,r): route_km[r] + train_cum_km[t]
             for t in trains for r in routes
             }

    for r in routes:
        model.Minimize(sum(assignments[t,r]*score[t,r] for t in trains))

    status = solver.Solve(model)
    assert status in [cp_model.FEASIBLE, cp_model.OPTIMAL]
 
    for t in trains:
        t_routes = [r for r in routes if solver.Value(assignments[t, r])]
        print(f'Train {t} is assigned to route/s {t_routes} '
              f'with end of day cumulative kilometreage of '
              f'{solver.Value(day_end_km[t])}')
 
 
if __name__ == '__main__':
    main()

Train T32 is assigned to route/s ['R42', 'R45'] with end of day cumulative kilometreage of 24320
Train T11 is assigned to route/s ['R41', 'R44'] with end of day cumulative kilometreage of 24320
Train T38 is assigned to route/s ['R43'] with end of day cumulative kilometreage of 24310
Train T28 is assigned to route/s ['R16'] with end of day cumulative kilometreage of 1200
Train T15 is assigned to route/s ['R32'] with end of day cumulative kilometreage of 800
Train T24 is assigned to route/s ['R11'] with end of day cumulative kilometreage of 800
